<details>
<summary>Table of Contents</summary>

- [1. Pretrain](#1-pretrain)
- [2. Finetune](#2-finetune)

</details>



This notebook demonstrates experiments on 5 countries with self-supervised learning. 
All parameters and settings are taken from the paper.

In [1]:
import pandas as pd
import subprocess
import os
from utils.helper import extract_metrics_from_output

# 1. Pretrain 

Tracking loss - MAE.

In [2]:
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
pred_lens = ["24", "96", "168"]

for country in countries:
    for pred_len in pred_lens:
        # Just use the same parameters as from the paper
        params = {
            "--dset": country,
            "--mask_ratio": 0.4,
            "--scaler_type": "minmax",
            "--n_epochs_pretrain": 100,
            "--target_points": pred_len
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_pretrain.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        # Execute the command
        print(f"==========Running command for {country}, pred_len {pred_len}:==========")
        !{command}


==========Running command for DE, pred_len 24:==========
args: Namespace(dset_pretrain='DE', context_points=512, target_points=24, batch_size=64, num_workers=0, features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=512, dropout=0.2, head_dropout=0.2, mask_ratio=0.4, n_epochs_pretrain=100, lr=0.0001, pretrained_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
suggested_lr 0.00017073526474706903
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
          epoch     train_loss     valid_loss           time
Better model found at epoch 0 with valid_loss value: 0.9374965442536594.
              0       0.981123       0.937497          00:08
Better model found at epoch 1 with valid_loss value: 0.9312531101717066.
              1       0.944243       0.931253      

# 2. Finetune

Linear probing + full fine-tuning.

In [2]:
# In case you just want finetune, please uncomment below
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
pred_lens = ["24", "96", "168"]

# List to store metrics for the DataFrame
data = []

for country in countries:
    for pred_len in pred_lens:
        # Define the parameters for each country and pred_len
        params = {
            "--dset": country,
            "--linear_prob_finetune": 1,
            "--pretrained_model": f"saved_models/{country}/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth",
            "--scaler_type": "minmax",
            "--n_epochs_finetune": 10, # parameters from paper. Though, original code did not have this implementation.
            "--n_epochs_linear_probe": 20,
            "--target_points": pred_len  
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_finetune.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        print(f"==========Running command for {country}, pred_len {pred_len}:==========")

        # Run the command and capture the output
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Capture the output in real-time
        output = []
        for line in process.stdout:
            output.append(line)
            print(line, end='')  # Print in the .ipynb cell

        # Wait for the process to complete
        process.stdout.close()
        process.wait()

        try:
            # Extract metrics from the output using the function
            metrics = extract_metrics_from_output(output, if_supervised=False)
            mse, rmse, mae = metrics[0]

            # Append the results to the data list
            data.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
            })
        except ValueError as e:
            print(f"\nError: {e} for {country}, pred_len {pred_len}")

==========Running command for DE, pred_len 24:==========
args: Namespace(linear_prob_finetune=1, is_finetune=0, is_linear_probe=0, n_epochs_linear_probe=20, dset_finetune='DE', context_points=512, target_points=24, batch_size=64, num_workers=0, scaler='standard', features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=256, dropout=0.2, head_dropout=0.2, n_epochs_finetune=10, lr=0.0001, pretrained_model='saved_models/DE/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth', finetuned_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 533528
check unmatched_layers: ['backbone.encoder.layers.0.ff.0.weight', 'backbone.encoder.layers.0.ff.0.bias', 'backbone.encoder.layers.0.ff.3.weight', 'backbone.encoder.layers.1.ff.0.weight', 'backbone.encoder.layers.1.ff.0.bias

In [6]:
df_self_sup = pd.DataFrame(data)
df_self_sup.set_index(['Country', 'Pred_len'], inplace=True)

# Path if not exists
path = "/results/PatchTST/"
if os.path.exists(path):
    os.makedirs(path)

# Save the results to a CSV file
df_self_sup.to_csv("finetune_results.csv", index=True)

df_self_sup = df_self_sup.round(4)
df_self_sup

MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0210  0.1448  0.0887
        96        0.0363  0.1906  0.1269
        168       0.0379  0.1947  0.1319
GB      24        0.0242  0.1557  0.0991
        96        0.0403  0.2007  0.1385
        168       0.0431  0.2076  0.1441
ES      24        0.0098  0.0990  0.0607
        96        0.0185  0.1358  0.0871
        168       0.0213  0.1458  0.0952
FR      24        0.0100  0.1001  0.0554
        96        0.0201  0.1419  0.0813
        168       0.0212  0.1455  0.0854
IT      24        0.0102  0.1008  0.0590
        96        0.0184  0.1355  0.0821
        168       0.0203  0.1426  0.0875

In [ ]:
import os
# Convert the list of dictionaries into a DataFrame
df_self_sup = pd.DataFrame(data)

# Set MultiIndex
df_self_sup.set_index(['Country', 'Pred_len'], inplace=True)
df_self_sup = df_self_sup.sort_index().round(4)

path = "/results/PatchTST/"
if os.path.exists(path):
    os.makedirs(path)


# Save the results to a CSV file
df_self_sup.to_csv("finetune_results.csv", index=True)
df_self_sup

In [8]:
df_self_sup.to_csv("finetune_results.csv", index=False)


In [16]:
import numpy as np
import pandas as pd
coun = np.repeat(["DE", "GB", "ES", "FR", "IT"], 3)
pred = np.array([24, 96, 168]*5)

df = pd.DataFrame({'Country': coun, 'Pred_len': pred})
df_2 = pd.read_csv("finetune_results.csv")
final = pd.concat([df, df_2], axis=1)
final

,Country,Pred_len,MSE,RMSE,MAE
0,DE,24,0.0210,0.1448,0.0887
1,DE,96,0.0363,0.1906,0.1269
2,DE,168,0.0379,0.1947,0.1319
3,GB,24,0.0242,0.1557,0.0991
4,GB,96,0.0403,0.2007,0.1385
5,GB,168,0.0431,0.2076,0.1441
6,ES,24,0.0098,0.0990,0.0607
7,ES,96,0.0185,0.1358,0.0871
8,ES,168,0.0213,0.1458,0.0952
9,FR,24,0.0100,0.1001,0.0554


In [17]:
final.index

RangeIndex(start=0, stop=15, step=1)

In [18]:
final.to_csv("finetune_results.csv", index=False)

# Test on bugs for Italy

In [ ]:
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
pred_lens = ["24", "96", "168"]

for country in countries:
    for pred_len in pred_lens:
        # Just use the same parameters as from the paper
        params = {
            "--dset": country,
            "--mask_ratio": 0.4,
            "--scaler_type": "minmax",
            "--n_epochs_pretrain": 100,
            "--target_points": pred_len
            }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_pretrain.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        # Complete command
        !{command}

args: Namespace(dset_pretrain='IT', context_points=512, target_points=96, batch_size=64, num_workers=0, features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=512, dropout=0.2, head_dropout=0.2, mask_ratio=0.4, n_epochs_pretrain=100, lr=0.0001, pretrained_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
suggested_lr 0.0002477076355991711
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
          epoch     train_loss     valid_loss           time
Better model found at epoch 0 with valid_loss value: 0.8967356272884459.
              0       0.938521       0.896736          00:09
Better model found at epoch 1 with valid_loss value: 0.8881698471826688.
              1       0.896467       0.888170          00:09
Better model found at epoch 2 with valid_loss va

In [6]:
# IT IS BETTER!
# Define parameters in a dictionary
params = {
    "--dset": "IT",
    "--linear_prob_finetune": 1,
    "--pretrained_model": "saved_models/IT/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain30_mask0.4_model1.pth",    
    "--scaler_type": "minmax",
    "--n_epochs_finetune": 10,
    "--n_epochs_linear_probe": 20
}

# Build the command string
command = "python PatchTST-main/PatchTST_self_supervised/patchtst_finetune.py "

# Add parameters to the command
for key, value in params.items():
    if value is not None:
        command += f"{key} {value} "
    else:
        command += f"{key} "  # Add flags with no value

# Run the command with !
!{command}

args: Namespace(linear_prob_finetune=1, is_finetune=0, is_linear_probe=0, n_epochs_linear_probe=20, dset_finetune='IT', context_points=512, target_points=96, batch_size=64, num_workers=0, scaler='standard', features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=256, dropout=0.2, head_dropout=0.2, n_epochs_finetune=10, lr=0.0001, pretrained_model='saved_models/IT/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain30_mask0.4_model1.pth', finetuned_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 920672
check unmatched_layers: ['backbone.encoder.layers.0.ff.0.weight', 'backbone.encoder.layers.0.ff.0.bias', 'backbone.encoder.layers.0.ff.3.weight', 'backbone.encoder.layers.1.ff.0.weight', 'backbone.encoder.layers.1.ff.0.bias', 'backbone.encoder.layers.1.ff.3.weight', 'backbone.enco